In [ ]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules.block import C2f

# ==================== CBAM Modülleri ====================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes,    1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        return self.sigmoid(avg_out + max_out) * x

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = x.mean(dim=1, keepdim=True)
        max_out, _ = x.max(dim=1, keepdim=True)
        return self.sigmoid(self.conv(torch.cat([avg_out, max_out], dim=1))) * x

class CBAM(nn.Module):
    def __init__(self, channels, ratio=8, kernel_size=7):
        super().__init__()
        self.channel_attention = ChannelAttention(channels, ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        # Eğitim sırasında her forward’da bu mesajı göreceksin
        print("✅ CBAM forward tetiklendi")
        return x

# ==================== CBAM Entegrasyonu ====================
def integrate_cbam_to_yolo(yolo_model):
    """
    Model katmanları arasında C2f bloklarını bulup sonuna CBAM ekler,
    doğru kanal sayısını layer.cv2.conv.out_channels ile alır.
    """
    layers = yolo_model.model.model  # ultralytics iç katman listesi
    for idx, layer in enumerate(layers):
        if isinstance(layer, C2f):
            # C2f içindeki cv2, bir Conv modülü; onun .conv.out_channels ile alıyoruz
            ch = layer.cv2.conv.out_channels
            layers[idx] = nn.Sequential(layer, CBAM(ch))
            print(f"✅ CBAM eklendi: C2f @ idx {idx}, channels={ch}")
    return yolo_model

# ==================== Ana Fonksiyon ====================
def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Önceden eğitilmiş segmentation modelini yükle
    yolo = YOLO('yolo11m-seg.pt')
    yolo.model.to(device)

    # Tüm C2f bloklarına CBAM ekle
    yolo = integrate_cbam_to_yolo(yolo)

    # Eğitim
    yolo.train(
        data='/teamspace/studios/data/data.yaml',
        epochs=75,
        imgsz=640,
        batch=16,
        project='YOLOv11m_CBAM_InstSeg',
        name='exp1',
        device=device,
        exist_ok=True,
        optimizer='AdamW',
        lr0=0.001
    )

if __name__ == '__main__':
    main()
